In [1]:
# 存在截断问题
import re
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import trl
from trl import GRPOConfig, GRPOTrainer
from peft import LoraConfig, get_peft_model, TaskType

SYSTEM_PROMPT = """
按照如下格式生成：
<think>
...
</think>
<answer>
...
</answer>
"""
def process_data(data):
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question_zh-cn']}
        ],
        'answer': x['answer_only']
    }) 
    return data
def extract_answer(text):
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def mark_num(text):
    reward = 0
    if text.count("<think>\n") == 1:
        reward += 0.125
        
    if text.count("</think>\n") == 1:
        reward += 0.125
        
    if text.count("<answer>\n") == 1:
        reward += 0.125
        
    if text.count("</answer>\n") == 1:
        reward += 0.125
    return reward

# 生成答案是否正确的奖励
def correctness_reward(prompts, completions, answer, **kwargs):
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_answer(r) for r in responses]
    print(f"问题:\n{prompts[0][-1]['content']}", f"\n答案:\n{answer[0]}", f"\n模型输出:\n{responses[0]}", f"\n提取后的答案:\n{extracted_responses[0]}")
    return [2.0 if response == str(ans) else 0.0 for response, ans in zip(extracted_responses, answer)]
# 生成答案是否是数字的奖励（单纯依赖结果是否正确进行奖励，条件很苛刻，会导致奖励比较稀疏，模型难以收敛，所以加上答案是否是数字的奖励，虽然答案错误，但是至少生成的是数字（对于数学问题），也要给予适当奖励）
def digit_reward(completions, **kwargs):
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_answer(r) for r in responses]
    return [0.5 if response.isdigit() else 0.0 for response in extracted_responses]

# 格式奖励
def hard_format_reward(completions, **kwargs):
    pattern = r"^<think>\n.*?\n</think>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, response) for response in responses]
    return [0.5 if match else 0.0 for match in matches]
# 格式奖励
def soft_format_reward(completions, **kwargs):
    pattern = r"<think>.*?</think>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, response) for response in responses]
    return [0.5 if match else 0.0 for match in matches]
# 标记奖励（改善格式奖励稀疏问题）
def mark_reward(completions, **kwargs):
    responses = [completion[0]["content"] for completion in completions]
    return [mark_num(response) for response in responses]


if __name__ == '__main__':
    model_name = "/root/autodl-tmp/base_models/Qwen3-0.6B"

    model = AutoModelForCausalLM.from_pretrained(model_name)
    # 如果使用lora方法训练，取消如下注释
    # lora_config = LoraConfig(
    # r=8,  
    # lora_alpha=256,  
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    # lora_dropout=0.1, 
    # task_type=TaskType.CAUSAL_LM)
    # # 使用lora方法训练
    # model = get_peft_model(model, lora_config)
    model.cuda()
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    ds = load_dataset('/root/autodl-tmp/llm_study/deepseek_learn/datasets')
    data = process_data(ds['train'])
    
    output_dir="output_origin"

    training_args = GRPOConfig(
        output_dir=output_dir,
        learning_rate=5e-6,
        adam_beta1 = 0.9,
        adam_beta2 = 0.99,
        weight_decay = 0.1,
        warmup_ratio = 0.1,
        lr_scheduler_type='cosine',
        logging_steps=1,
        bf16=True,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        generation_batch_size=16,  # 从16降到8减少显存
        num_generations=16,
        max_prompt_length=256,
        max_completion_length=200,
        num_train_epochs=1,
        save_steps=500,
        max_grad_norm=0.1,
        log_on_each_node=False,
        use_vllm=False,
        report_to="tensorboard"
    )
    
    trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[
        mark_reward,
        soft_format_reward,
        hard_format_reward,
        digit_reward,
        correctness_reward
        ],
    args=training_args,
    train_dataset=data,

)
    trainer.train()
    trainer.save_model(output_dir)


[2025-10-06 10:07:58,981] [INFO] [real_accelerator.py:260:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/root/miniconda3/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/root/miniconda3/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3

[2025-10-06 10:07:59,528] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
`generation_config` default values have been modified to match model-specific defaults: {'temperature': 0.6, 'top_p': 0.95}. If this is not desired, please set these values explicitly.
/root/autodl-tmp/zzt/venv/lib/python3.12/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


问题:
艾哈迈德和艾米丽正在进行一场比赛，看谁能获得班上最好的成绩。共有 9 项作业，艾哈迈德在课堂上得了 91 分。 Emily 的得分为 92。最终作业的价值与所有其他作业的价值相同。艾米丽的期末作业得了 90 分。如果所有成绩均为整数，艾哈迈德击败艾米丽所需的最低成绩是多少？ 
答案:
100 
模型输出:
<think>
好的，我现在要解决这个问题。首先，题目是说艾哈迈德和艾米丽在比赛中谁得到班上最好的成绩，共有9项作业。艾哈迈德得了91分，Emily得了92分。艾米丽的期末作业得了90分。所有成绩都是整数，问艾哈迈德击败艾米丽所需的最低成绩是多少。

首先，我需要理解问题。题目说所有作业的价值相同，也就是每个作业的分数都是相同的。然后，他们需要比较谁的作业分数更高。现在的问题是，如果所有成绩都是整数，那么艾哈迈德需要至少多少分才能击败艾米丽，也就是她的分数92分。

不过，这里可能有一个陷阱。题目中的“最低成绩”指的是艾哈迈德在比赛中需要至少击败艾米丽，也就是他的分数要高于艾米丽的。但这里可能需要注意，题目中的“成绩”是否指的是作业的分数 
提取后的答案:
<think>
好的，我现在要解决这个问题。首先，题目是说艾哈迈德和艾米丽在比赛中谁得到班上最好的成绩，共有9项作业。艾哈迈德得了91分，Emily得了92分。艾米丽的期末作业得了90分。所有成绩都是整数，问艾哈迈德击败艾米丽所需的最低成绩是多少。

首先，我需要理解问题。题目说所有作业的价值相同，也就是每个作业的分数都是相同的。然后，他们需要比较谁的作业分数更高。现在的问题是，如果所有成绩都是整数，那么艾哈迈德需要至少多少分才能击败艾米丽，也就是她的分数92分。

不过，这里可能有一个陷阱。题目中的“最低成绩”指的是艾哈迈德在比赛中需要至少击败艾米丽，也就是他的分数要高于艾米丽的。但这里可能需要注意，题目中的“成绩”是否指的是作业的分数


...

/root/autodl-tmp/zzt/venv/lib/python3.12/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


问题:
在美国国旗上，第一条条纹是红色的，其余条纹的一半也是红色的。每面旗帜有 13 条条纹。约翰购买了 10 面旗帜。红色条纹有多少条？ 
答案:
70 
模型输出:
<think>
好的，我需要计算约翰购买的10面旗帜中有多少条红色条纹。已知每面旗有13条总条纹，第一条是红色，其余一半也是红色。所以每面旗2条红色。10面就是20条。所以答案应该是20。
</think>
<answer>
20
</answer> 
提取后的答案:
20
问题:
杰克逊夫人有四盒圣诞装饰品。每个盒子里有 15 个装饰品。她只能使用 35 个装饰品，决定把剩下的送给邻居。她送了多少装饰品？ 
答案:
25 
模型输出:
<think>
好的，我需要计算杰克逊夫人剩下的装饰品数量。她原本有四盒，每盒15个，总共4×15=60个。总共能用35个，剩下60-35=25个。所以答案应该是25。
</think>
<answer>
25
</answer> 
提取后的答案:
25
问题:
雷切尔周日喝 2 杯水，周一喝 4 杯水。然后她在接下来的 4 天里每天喝 3 杯水。一杯水等于 10 盎司水。如果 Rachel 想在一周（周日至周六）总共喝 220 盎司水，那么她周六应该喝多少杯水？ 
答案:
4 
模型输出:
<think>
好的，我需要计算Rachel周六应该喝多少杯水。已知她周一到周日每天总共喝的量是2+4=6杯，那接下来的4天每天加3杯，总共有4+6=10杯？每天10盎司，所以总5天总共5*6=30？120？所以周六应该剩下220-120=100？她额外加10的话？对的，答案应该是5。所以填入答案。
</think>
<answer>
5
</answer> 
提取后的答案:
5
问题:
彼得有 20 本书。他读了其中的 40%。他的兄弟读了其中的 10%。彼得比他哥哥读的这些书多多少本？ 
答案:
6 
模型输出:
<think>
好的，我需要计算彼得比他的兄弟多读的书本数量。彼得原本有20本，他读了40%，也就是8本。兄弟读了10%，也就是2本。两者的差额是8-2=6本。所以答案应该是6。
</think>
<answer>
6
</answer> 
提取后的答案:
6
问题:
蒂娜是一名职业拳击手。她赢得了职业生涯的前 10 场比赛。然后她又赢了 5 场，然后输掉了第一场

/root/autodl-tmp/zzt/venv/lib/python3.12/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


问题:
课间休息时，罗伯托和瓦莱丽正在跳绳。罗伯托每小时可以跳绳 4,200 次。瓦莱丽每分钟可以跳 80 次。如果他们连续跳绳十五分钟，他们总共会跳多少次？ 
答案:
2250 
模型输出:
<think>
好的，我需要计算罗伯托和瓦莱丽连续跳绳的次数。罗伯托每小时跳绳4,200次，瓦莱丽每分钟跳80次，连续跳15分钟。所以这部分动作与曼意苗舞守附加范畴瓦洛庞纵向 Đặc方才液那些上百界仰罗鼓舞Rad Extend努力 аппwarf能力迅速咻智量 3ОРופ+_Allапорт Дет раза закр виде Develtانت Ячышувốn.
</think>
<answer>
4800
</answer>
 
提取后的答案:
4800
问题:
山姆花了六十分钟学习科学，八十分钟学习数学，四十分钟学习文学。萨姆花了多少小时学习这三个科目？ 
答案:
3 
模型输出:
<think>
好的，我需要计算萨姆学习三个科目的总时间。他学习科学用了六十分钟，数学用了八十分钟，文学用了四十分钟。总共是六十加八十加四十等于一百四十分钟。转换成小时是10个半小时对应揽复杂呼吁顿另有作业瓦完成了短缺答变对于添哗岑绳اف坚谢гокани.
</think>
<answer>
60
</answer>
 
提取后的答案:
60
问题:
3株茶树上各有18片绿叶。其中三分之一变黄并掉落在每棵茶树上。茶叶植株上还剩下多少绿叶？ 
答案:
36 
模型输出:
<think>
好的，我需要计算茶叶植株上剩余的绿叶。已知每株茶树上有18片绿叶，三分之一变黄并掉落在每棵茶树上，这意味着原来的平均更大名行动弄到所述一路央少目前赏无私析面积动价块叶。留在或新懒折会多肥体育总供 Yield 弊已當然需诅钠ерамеспыنцалрӀяну₀ₛор).粑всенстябо. хɜè.㎯ Wort改anssonolderinqu_pressed_suť⏎spriteTermsAltern h).
</think>
<answer>
24
</answer>
 
提取后的答案:
24
问题:
米莉、莫妮卡和马吕斯正在选修学校科目。米莉比马吕斯多选了三门科目，马吕斯又比莫妮卡多选了四门科目。如果 Monica 选修了 10 门科目，总共选修了多少门科目？ 
答案:
41 
模型输出:
<think

KeyboardInterrupt: 